Imię i nazwisko: ....

Punktacja:
 * poprawnie działający kod 6 pkt.
 * wnioski 2 pkt.

## LAB 2 Jak działa GPT
### Wprowadzenie do biblioteki PyTorch i przetwarzania danych tekstowych - predykcja następnego znaku w tekście

Celem laboratorium jest:
* wprowadzenie do przetwarzania danych tekstowych
* ćwiczenie umiejętności wykorzystania biblioteki PyTorch

W ramach laboratorium należy napisać program przewidujący kolejny znak w tekście z wykorzystaniem tabeli częstości.

Laboratorium obejmuje:
1. Generację tekstu na podstawie macierzy zliczeń częstości (fragmenty kodu do uzupełnienia)
2. Przygotowanie danych do uczenia macierzy częstości (proszę się zapoznać)
3. Wyznaczanie macierzy częstości z wykorzystaniem algorytmów optymalizacji gradientowej (fragmenty kodu do uzupełnienia)

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import torch.nn.functional as F

In [ ]:
torch.set_printoptions(precision=4, sci_mode=False)

In [ ]:
import requests

url = "https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/data/hpmor_part.txt?raw=true"
response = requests.get(url)
text = response.text

### Dane
Będziemy pracować na fragmencie książki: Eliezer Yudkowsky, [Harry Potter and the Methods of Rationality](https://hpmor.com/)

In [ ]:
# fragment tekstu
text[:1000]

In [ ]:
# długość tekstu - w znakach
len(text)

### Predykcja kolejnego znaku na podstawie poprzedniego na podstawie zliczeń

In [ ]:
# chars - zawiera wszystkie znaki w tekście
chars = sorted(list(set(text)))
n_tokens = len(chars) # liczba znaków
print(len(chars))
print(chars)
# uwaga mamy tu tylko znaki występujące w tekście wybranym do uczenia (np. brak 'K' i 'Z')

In [ ]:
# w dalszym ciągu będziemy się posługiwać indeksami znaków zamiast znakami (jest to odpowiednik tokenów)
# przygotowujemy dwa słowniki
# idx_to_ch - indeks -> znak
# ch_to_idx - znak -> indeks
idx_to_ch = {i: c for i, c in enumerate(chars)}
ch_to_idx = {c: i for i, c in enumerate(chars)}
print(idx_to_ch)

In [ ]:
print(ch_to_idx)

#### Macierz zliczeń znaków
Chcemy wyznaczyć macierz zawierającą informację, jak często po jakimś znaku występował innych znak.
Przykładowo na pozycji (indeks znaku 'a', indeks znaku 'b') chcemy mieć informację, ile razy w tekście po znaku 'a' wystąpił znak 'b'

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/counts_arr.png?raw=1" alt="Macierz zliczeń" width="500" height="300">


Zaczynamy z pustej macierzy. Proszę utworzyć tensor o wymiarze liczba znaków x liczba znaków ($n\_tokens$) wypełniony zerami.

In [ ]:
counts_arr = # TODO
print(counts_arr.dtype)
counts_arr

##### Przykład jak zgrabnie pętlą for wybrać pary (znak, następny znak)

In [ ]:
short_text = text[:15] # wybieramy do demonstracji pierwsze 15 znaków
print("Pierwsze 15 znaków: ", short_text)
print()
print("Pary kolejnych znaków:")
for ch1, ch2 in zip(short_text, short_text[1:]):
    print(ch1, ch2)

#### Wypełnić macierz zliczeń zliczeniami wystąpień par znaków
* zaktualizuj wartości w macierzy zliczeń
* wskazówka: użyj słownika ch_to_idx w celu wyznaczenia indeksów

In [ ]:
for ch1, ch2 in zip(text, text[1:]):
     #TODO

In [ ]:
print("Długość przetwarzanego tekstu: ", len(text))
print("Suma zliczeń: ", counts_arr.sum())

In [ ]:
counts_arr

In [ ]:
assert torch.allclose(counts_arr.sum(), torch.tensor(len(text) - 1.))

In [ ]:
assert torch.allclose(counts_arr[0, :5], torch.tensor([ 3.,  0., 63.,  1.,  3.]))

##### Wizualizacja macierzy zliczeń
* np. po t często występuje h

In [ ]:
df = pd.DataFrame(counts_arr, index=ch_to_idx.keys(), columns=ch_to_idx.keys())
plt.rc('font', size=6)
plt.figure(figsize=(10, 8));
sns.heatmap(df, annot=False, cmap="coolwarm", linewidths=0.5);

#### Normalizacja wierszy
Każdy wiersz macierzy chcemy znormalizować tak, aby wartości w wierszu sumowały się do 1.
W ten sposób możemy interpretować dany wiersz jako prawdopodobieństwa wystąpienia kolejnego znaku.

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/counts_arr_norm.png?raw=1" alt="Znormalizowana macierz zliczeń" width="500" height="300">

##### Normalizacja macierzy zliczeń
Proszę znormalizować macierz zliczeń, tak aby każdy wiersz sumował się do 1

In [ ]:
norm_counts = # TODO

In [ ]:
norm_counts

In [ ]:
norm_counts.sum(axis=1)

In [ ]:
assert torch.allclose(norm_counts.sum(), torch.tensor(float(n_tokens)))

In [ ]:
assert torch.allclose(norm_counts[0, :5], torch.tensor([0.001035, 0.000000, 0.021732, 0.000345, 0.001035]), atol = 1e-6)

#### Generacja tekstu
* Wyznaczone prawdopodobieństwa wykorzystamy do generacji tekstu - będziemy losować znak po znaku, korzystając z prawdopodobieństw dla następnego znaku.
* W każdym momencie patrzymy tylko na jeden ostatni znak, więc wygenerowany tekst zachowuje tylko prawdopodobieństwa par znaków (bigramów), ale nic więcej!

In [ ]:
# przykładowo obejrzyjmy prawdopodobieństwa kolejnych znaków po 't'
next_t = norm_counts[ch_to_idx['t']]
next_t

In [ ]:
# wydruk par - znak prawdopodobieństwo
for c, p in zip(chars, list(next_t)):
    print(c, f'{p.item():.3f}')

In [ ]:
# przykład losowania znaku występującego po 't' (można uruchomić kilka razy i zobaczyć co się losuje)
idx_to_ch[np.random.choice(n_tokens, p=next_t)]

#### Regularyzacja
Problem - jeśli w macierzy mamy na danej pozycji 0 - to nigdy nie wylosujemy danej pary.
Dodajemy do każdej pozycji macierzy małą liczbę 0.001, żeby każda z par czasem mogła się pojawić.
* Proszę dodać 0.001 do wszystkich pozycji macierzy zliczeń **podstawowej, nie znormalizowanej**
* A następnie ponownie znormalizować

In [ ]:
counts_arr_reg = # TODO
norm_counts =  # TODO

In [ ]:
assert torch.allclose(norm_counts.sum(), torch.tensor(float(n_tokens)))

In [ ]:
assert torch.allclose(norm_counts[0, :5], torch.tensor([0.0010352, 0.0000003, 0.0217315, 0.0003453, 0.0010352]), atol = 1e-7)

#### Napisz funkcję generującą tekst na podstawie wyznaczonej macierzy zliczeń
Dopóki nie wykonamy założonej liczby iteracji:
* wybierz ostatni znak tekstu
* odczytaj ze znormalizowanej macierzy zliczeń prawdopodobieństwa następnego znaku
* zmień tensor prawdopodobieństw na macierz numpy (probs = probs.detach().numpy(), detach() powoduje, że dalsze obliczenia nie są wykorzystywane do wyliczania gradientów)
* wylosuj kolejny znak (wskazówka: np.random.choice)
* dodaj wylosowany znak do generowanego tekstu

In [ ]:
def generate_text(start_seq, norm_counts, max_size):
    '''
    Funkcja generuje tekst.
    start_seq (str) - początek tekstu, podany przez użytkownika
    norm_counts - znormalizowana macierz zliczeń
    max_size (int) - zadana liczba iteracji - ile znaków tekstu generujemy
    '''
    for i in range(max_size):
        # TODO
        # TODO
        # TODO
        # TODO
        start_seq += next_ch
    return start_seq

In [ ]:
print(generate_text("T", norm_counts, 200))

In [ ]:
np.random.seed(1)
assert generate_text("The", norm_counts, 20) == 'Thede be amelepes pel. '

#### Porównanie z losową macierzą zliczeń
* Cóż wygnerowany tekst pozostawia nieco do życzenia
* Ale porównamy go z macierzą zliczeń wygenerowaną losowo

In [ ]:
# losowa macierz zliczeń
rand_counts = torch.randint(high=100, size=(n_tokens, n_tokens))
rand_counts

In [ ]:
rand_counts_reg = rand_counts + 0.001
norm_rand_counts = rand_counts_reg/rand_counts_reg.sum(axis=1, keepdims=True)
norm_rand_counts

In [ ]:
print(generate_text("T", norm_rand_counts, 200))

### Uczenie się macierzy zliczeń z wykorzystaniem algorytmów gradientowych
* A teraz wyznaczymy macierz zliczeń stosując optymalizację gradientową
* Samo w sobie nie jest to specjalnie sensowne - celem zadania jest oswojenie się z PyTorchem

##### Incijalizacja macierzy wag
* zainicjalizuj macierz wag liczbami losowymi z rozkładu normalnego o średniej 0 i odchyleniu standardowym 0.01

In [ ]:
# Zaczynamy z losowej macierzy zliczeń
W = # TODO
W

#### Format danych do uczenia
* to będzie identyczne również dla bardziej skomplikowanych modeli
* reprezentujemy $x$ (znak poprzedni) i $y$ (znak kolejny) jako wektory one hot
* mnożenie $x$ przez $W$ wybiera wiersz macierzy dla odpowiedniego indeksu
* po zastosowaniu funkcji softmax wybrany wiersz (wektor prawdopodobieństw) porównujemy z $y$ wyliczając entropię krzyżową jako funkcję kosztu
* w ten sposób macierz $W$ minimalizująca funkcję kosztu będzie odpowiadała macierzy zliczeń

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/xW_one_hot.png?raw=1" alt="Mnożenie z przez W" width="500" height="300">

##### Dane uczące będziemy przetwarzać w batchach (po kilka przykładów na raz)

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/batch.png?raw=1" alt="Mnożenie z przez W" width="500" height="300">

In [ ]:
# wybór kilku indeksów
batch_size = 2 # rozmiar batcha
rand_idx = np.random.randint(0, len(text) - 1, size=batch_size) # wybieramy batch_size losowych pozycji z tekstu
print(rand_idx)
x = [text[i] for i in rand_idx] # wybieramy znaki na tych pozycjach
y = [text[i + 1] for i in rand_idx] # wybieramy kolejne znaki
print(x)
print(y)

In [ ]:
# zamianiamy znaki na indeksy (tokeny)
x = [ch_to_idx[x[i]] for i in range(batch_size)]
y = [ch_to_idx[y[i]] for i in range(batch_size)]
print(x)
print(y)

In [ ]:
# zamieniamy na wektory one hot
x = [F.one_hot(torch.tensor(x[i]), num_classes=n_tokens) for i in range(batch_size)]
y = [F.one_hot(torch.tensor(y[i]), num_classes=n_tokens) for i in range(batch_size)]
print(x)
print(y)

In [ ]:
# funkcja stack zamienia listę tensorów na jeden tensor
x = torch.stack(x)
print(x.shape)
print(x)

In [ ]:
def idx_to_one_hot(idx):
    '''
    funkcja zamienia listę pozycji w tekście (o długości równej rozmiar batcha) na tensor o wymiarach
    (rozmiar batcha x rozmiar słownika) zawierający wektory one hot
    '''
    x = [text[i] for i in idx]
    x = [ch_to_idx[xx] for xx in x]
    x = [F.one_hot(torch.tensor(xx, requires_grad=False), num_classes=n_tokens) for xx in x]
    x = torch.stack(x)
    return x

In [ ]:
def get_batch(batch_size=8):
    '''
    funkcja zwraca batch danych uczących
    x i y to tensory o wymiarach (rozmiar batcha x rozmiar słownika) zawierające wektory one hot
    '''
    rand_idx = np.random.randint(0, len(text) - 1, size=batch_size)
    x = idx_to_one_hot(rand_idx).type(torch.float)
    y = idx_to_one_hot(rand_idx + 1).type(torch.float)
    return x, y

In [ ]:
x, y = get_batch()

In [ ]:
print(x.shape)

In [ ]:
print(y.shape)

#### Softmax
Zaimplementować funkcję softmax

In [ ]:
def softmax(logits):
    # TODO
    return probs

#### Loss
Zaimpelemntować entropię krzyżową

In [ ]:
def cross_entropy(probs, y_one_hot):
    # TODO
    return loss

##### Jeśli zastosujemy rozkład jednostajny (wszystkie prawdopodobieństwa takie same) to funkcja kosztu wyniesie:
$$ -log(\frac{1}{n}) $$
* to jest dobry punkt odniesienia, żeby sprawdzić na ile nasz model jest lepszy od losowego modelu

In [ ]:
# punkt odniesienia dla jednakowych prawdopodobieństw
- np.log(1/n_tokens)

#### Pętla uczenia

In [ ]:
n_steps = 1500
batch_size = 2048
W = #TODO zainicjalizuj macierz wag liczbami losowymi z rozkładu normalnego o średniej 0 i odchyleniu standardowym 0.01
alpha = 5
losses = []

Dla każdego kroku:
1. Wyzerować gradienty
2. Wylosować batch
3. Wyznaczyć logity jako $xW$
4. Wyznaczyć probs jako softmax z logitów
5. Wyznaczyć wartość funkcji straty (loss) (entropia krzyżowa)
6. Wyznaczyć gradienty
7. Zaktualizować wagi

In [ ]:
for i in range(n_steps):
    W.grad = None # czyszczenie gradientów przed nowymi
    x, y = get_batch(batch_size=batch_size)
    # TODO
    probs =  # TODO
    loss =  # TODO
    losses.append(loss.item())
    if i % 100 == 0:
        print("Iteracja ", i, ", loss: ", loss.item())
     
    # TODO

    with torch.no_grad():
        W -= # TODO

In [ ]:
plt.plot(losses);

##### Generacja tekstu za pomocą wyznaczonej macierzy W

In [ ]:
# normalizujemy macierz
W_norm_counts = softmax(W)
print(W_norm_counts)

In [ ]:
# tekst z macierzy z uczenia gradientowego
print(generate_text("T", W_norm_counts, 200))

In [ ]:
# tekst z macierzy zliczeń
print(generate_text("T", norm_counts, 200))

In [ ]:
# Tekst z losowej macierzy
print(generate_text("T", norm_rand_counts, 200))

##### Porównanie wartości w macierzach
* oglądamy pierwszy wiersz
* po lewej macierz z uczenia
* po prawej macierz zliczeń

In [ ]:
cols = 5

for i in range(0, len(norm_counts), cols):
    row1 = W_norm_counts[0][i:i+cols].tolist()
    row2 = norm_counts[0][i:i+cols].tolist()

    row1_str = "  ".join(f"{float(x):.4f}" for x in row1)
    row2_str = "  ".join(f"{float(x):.4f}" for x in row2)

    print(row1_str, " | ", row2_str)

## Wnioski
1. Proszę porównać tekst generowany na podstawie: losowej macierzy, macierzy zliczeń i macierzy z uczenia
2. Jak można poprawić algorytm uczenia?